# Weather information: Top 35 cities to visit in France  

In this notebook I :  
-  Retrieved the weather information for the [top 35 best cities in France](https://one-week-in.com/35-cities-to-visit-in-france/)
-  Defined the top 5 cities to visit based on the best weather for the next 7 days

## Import libraries

In [2]:
import requests
import pandas as pd
import numpy as np
import json

from src.cities_weather import req_gps_fn, req_weather_fn, req_to_df_fn
from src.cities_weather import explode_df_fn, explode2_df_fn, explodeall_df_fn

## Import data

In [3]:
# Import list of top 35 cities in France
filename = 'data/raw/top_35_cities_france.txt'
with open(filename) as file:
    top_cities_ls = [line.rstrip() for line in file]

print(len(top_cities_ls))
print(top_cities_ls[0:4])

35
['Mont Saint Michel', 'Saint-Malo', 'Bayeux', 'Le Havre']


## 1. GPS coordinates

Use the [nominatim API](https://nominatim.org/) to get latitute and longitude for each of the cities of interest 


### API request

In [4]:
# Use previously defined requests function  to retrieve response for top cities
api_gps=[req_gps_fn(x) for x in top_cities_ls]
print('API response length: ', len(api_gps))
api_gps[0]

API response length:  35


[{'place_id': 151486647,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 211285890,
  'boundingbox': ['48.6349172', '48.637031', '-1.5133292', '-1.5094796'],
  'lat': '48.6359541',
  'lon': '-1.511459954959514',
  'display_name': 'Mont Saint-Michel, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France',
  'place_rank': 20,
  'category': 'place',
  'type': 'islet',
  'importance': 0.755436556781574},
 {'place_id': 282955240,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 7360493,
  'boundingbox': ['46.7308335', '46.9105744', '-75.4379111', '-75.1592812'],
  'lat': '46.7798558',
  'lon': '-75.336261',
  'display_name': 'Mont-Saint-Michel, Antoine-Labelle, Laurentides, Québec, J0W 1P0, Canada',
  'place_rank': 16,
  'category': 'boundary',
  'type': 'administrative',
  'importance': 0.7301414688716277,
  '

In [5]:
# Create dataframe from request response
api_cities_df = req_to_df_fn(api_gps)
api_cities_df.head()

Number of API keys retrieved in request response:  12
List of dataframes per API result. Length:  35
Merged API requests dataframe shape:  (294, 12)


,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,place_rank,category,type,importance
0,151486647,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",20,place,islet,0.755437
1,282955240,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,7360493,"[46.7308335, 46.9105744, -75.4379111, -75.1592...",46.7798558,-75.336261,"Mont-Saint-Michel, Antoine-Labelle, Laurentide...",16,boundary,administrative,0.730141
2,65638299,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,5972710523,"[48.6359711, 48.6360711, -1.5115447, -1.5114447]",48.6360211,-1.5114947,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",18,natural,peak,0.600000
3,14994654,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,1581196576,"[43.8495943, 43.8496943, 7.0938427, 7.0939427]",43.8496443,7.0938927,"Mont Saint-Michel, Les Ferres, Grasse, Alpes-M...",18,natural,peak,0.600000
4,334476680,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,9581659906,"[43.897659, 43.897759, 7.3643673, 7.3644673]",43.897709,7.3644173,"Mont Saint-Michel, Nice, Alpes-Maritimes, Prov...",18,natural,peak,0.600000


In [6]:
# Keep copy of original df
api_cities_df_raw=api_cities_df
api_cities_df=api_cities_df_raw.copy()

### Delete duplicated alias

In [7]:
#Create a column containing search name of city (obtained from display_name column)
api_cities_df['Search_Name']=api_cities_df.display_name.str.split(',',expand=True)[0]
api_cities_df.head()

#Sort by entry with highest importance (given based on Wikipedia entries) and then delete duplicated Search Names
api_cities_df=api_cities_df.sort_values('importance', ascending=False).drop_duplicates(subset=['Search_Name'])
api_cities_df=api_cities_df.sort_values('importance', ascending=False).drop_duplicates(subset=['Search_Name'])
print('Shape of duplicate clean cities dataframe: ', api_cities_df.shape)
api_cities_df.head()

Shape of duplicate clean cities dataframe:  (72, 13)


,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,place_rank,category,type,importance,Search_Name
0,282321559,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,414459,"[43.4062445, 43.5858463, 4.2301364, 4.6444785]",43.4522771,4.4287172,"Saintes-Maries-de-la-Mer, Arles, Bouches-du-Rh...",16,boundary,administrative,0.972517,Saintes-Maries-de-la-Mer
1,283556983,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,17807753,"[48.6934951, 49.0134951, 2.1883915, 2.5083915]",48.8534951,2.3483915,"Paris, Île-de-France, France métropolitaine, 7...",15,place,city,0.941710,Paris
0,281716908,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,70279,"[43.4461058, 43.6259224, 5.2694745, 5.5063013]",43.5298424,5.4474738,"Aix-en-Provence, Bouches-du-Rhône, Provence-Al...",16,boundary,administrative,0.891085,Aix-en-Provence
0,282341149,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",16,boundary,administrative,0.822333,Le Havre
0,281822562,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,117858,"[46.1331804, 46.1908971, -1.2419231, -1.111097]",46.1591126,-1.1520434,"La Rochelle, Charente-Maritime, Nouvelle-Aquit...",16,boundary,administrative,0.801484,La Rochelle


### GPS coordinates dataframe

In [8]:
#The GPS coordinates are found under the keys lat and lon
cols_keep_ls=["Search_Name","lat","lon"]
GPS_df=api_cities_df.loc[:,cols_keep_ls]

print('Cities GPS dataframe shape: ', GPS_df.shape)
GPS_df.head(44)

Cities GPS dataframe shape:  (72, 3)


,Search_Name,lat,lon
0,Saintes-Maries-de-la-Mer,43.4522771,4.4287172
1,Paris,48.8534951,2.3483915
0,Aix-en-Provence,43.5298424,5.4474738
0,Le Havre,49.4938975,0.1079732
0,La Rochelle,46.1591126,-1.1520434
0,Lyon,45.7578137,4.8320114
0,Marseille,43.2961743,5.3699525
0,Strasbourg,48.584614,7.7507127
1,Toulouse,43.6007049,1.4241145487393414
0,Bormes-les-Mimosas,43.1572172,6.329253867921363


In [9]:
#When filtering df by  names from initial list of cities we loose some of them. Update name in top cities file and re-do API request
s=pd.Series(top_cities_ls)
print(s[~s.isin(GPS_df["Search_Name"])])

Series([], dtype: object)


In [10]:
# Final GPS info dataframe. Select search names in list of top cities
GPS_df=GPS_df[GPS_df["Search_Name"].isin(top_cities_ls)]
GPS_df.sort_values(by=['Search_Name'],inplace=True)
print(GPS_df.shape)
GPS_df.head()

(35, 3)


,Search_Name,lat,lon
1,Aigues Mortes,45.3314132,1.03014
0,Aix-en-Provence,43.5298424,5.4474738
1,Amiens,49.898700500000004,2.2724569947750464
0,Annecy,45.8992348,6.1288847
0,Ariège,42.9455368,1.4065544156065486


## 2. Weather data  
Use the [openweathermap API](https://openweathermap.org/api/one-call-api) and previously obtained GPS coordinates to retrieve weather information for the cities of interest

In [11]:
# Define key needed for use API (obtained after registration)
API_key = open("access_keys/API_Key_OpenWeather.txt", "r").read()

In [12]:
#Latitude and longitude form df as lists
lat=GPS_df["lat"].values.tolist()
lon=GPS_df["lon"].values.tolist()

#List comprehension to get request for all of the cities found (by latitude and longitude)
api_weath=[req_weather_fn(x,y, key=API_key) for x,y in zip(lat,lon)]

print('API weather results length: ', len(api_weath)) 
print('API weather keys length: ',len(api_weath[0])) 
api_weath[0]

API weather results length:  35
API weather keys length:  6


{'lat': 45.3314,
 'lon': 1.0301,
 'timezone': 'Europe/Paris',
 'timezone_offset': 7200,
 'current': {'dt': 1649613278,
  'sunrise': 1649567987,
  'sunset': 1649615654,
  'temp': 11.75,
  'feels_like': 10.55,
  'pressure': 1013,
  'humidity': 60,
  'dew_point': 4.25,
  'uvi': 0,
  'clouds': 75,
  'visibility': 10000,
  'wind_speed': 1.49,
  'wind_deg': 97,
  'wind_gust': 1.49,
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04d'}]},
 'daily': [{'dt': 1649588400,
   'sunrise': 1649567987,
   'sunset': 1649615654,
   'moonrise': 1649589840,
   'moonset': 1649558640,
   'moon_phase': 0.28,
   'temp': {'day': 13.71,
    'min': 1.02,
    'max': 16.15,
    'night': 9.96,
    'eve': 12.32,
    'morn': 1.02},
   'feels_like': {'day': 12.08, 'night': 8.96, 'eve': 11.15, 'morn': -0.95},
   'pressure': 1018,
   'humidity': 36,
   'dew_point': -1.39,
   'wind_speed': 2.76,
   'wind_deg': 128,
   'wind_gust': 7.22,
   'weather': [{'id': 803,
     'mai

In [13]:
# Create dataframe from request response
weather_df_raw = req_to_df_fn(api_weath)
weather_df_raw.head()

Number of API keys retrieved in request response:  6
List of dataframes per API result. Length:  35
Merged API requests dataframe shape:  (35, 7)


,lat,lon,timezone,timezone_offset,current,daily,alerts
0,45.3314,1.0301,Europe/Paris,7200,"{'dt': 1649613278, 'sunrise': 1649567987, 'sun...","[{'dt': 1649588400, 'sunrise': 1649567987, 'su...",NaN
0,43.5298,5.4475,Europe/Paris,7200,"{'dt': 1649613278, 'sunrise': 1649567057, 'sun...","[{'dt': 1649588400, 'sunrise': 1649567057, 'su...",NaN
0,49.8987,2.2725,Europe/Paris,7200,"{'dt': 1649613278, 'sunrise': 1649567321, 'sun...","[{'dt': 1649588400, 'sunrise': 1649567321, 'su...",NaN
0,45.8992,6.1289,Europe/Paris,7200,"{'dt': 1649613278, 'sunrise': 1649566723, 'sun...","[{'dt': 1649588400, 'sunrise': 1649566723, 'su...","[{'sender_name': 'METEO-FRANCE', 'event': 'Mod..."
0,42.9455,1.4066,Europe/Paris,7200,"{'dt': 1649613278, 'sunrise': 1649568065, 'sun...","[{'dt': 1649588400, 'sunrise': 1649568065, 'su...","[{'sender_name': 'METEO-FRANCE', 'event': 'Mod..."


In [14]:
#Drop current weather information. Keep only daily
weather_df=weather_df_raw.copy()
weather_df.drop('current',axis=1,inplace=True)

In [15]:
# First data explosion: on chosen column: daily
df_explode = explode_df_fn(weather_df, 'daily')
print(df_explode.shape)
df_explode.head()

(35, 13)


,lat,lon,timezone,timezone_offset,alerts,Today,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7
0,45.3314,1.0301,Europe/Paris,7200,NaN,"{'dt': 1649588400, 'sunrise': 1649567987, 'sun...","{'dt': 1649674800, 'sunrise': 1649654278, 'sun...","{'dt': 1649761200, 'sunrise': 1649740570, 'sun...","{'dt': 1649847600, 'sunrise': 1649826862, 'sun...","{'dt': 1649934000, 'sunrise': 1649913155, 'sun...","{'dt': 1650020400, 'sunrise': 1649999449, 'sun...","{'dt': 1650106800, 'sunrise': 1650085744, 'sun...","{'dt': 1650193200, 'sunrise': 1650172039, 'sun..."
0,43.5298,5.4475,Europe/Paris,7200,NaN,"{'dt': 1649588400, 'sunrise': 1649567987, 'sun...","{'dt': 1649674800, 'sunrise': 1649654278, 'sun...","{'dt': 1649761200, 'sunrise': 1649740570, 'sun...","{'dt': 1649847600, 'sunrise': 1649826862, 'sun...","{'dt': 1649934000, 'sunrise': 1649913155, 'sun...","{'dt': 1650020400, 'sunrise': 1649999449, 'sun...","{'dt': 1650106800, 'sunrise': 1650085744, 'sun...","{'dt': 1650193200, 'sunrise': 1650172039, 'sun..."
0,49.8987,2.2725,Europe/Paris,7200,NaN,"{'dt': 1649588400, 'sunrise': 1649567987, 'sun...","{'dt': 1649674800, 'sunrise': 1649654278, 'sun...","{'dt': 1649761200, 'sunrise': 1649740570, 'sun...","{'dt': 1649847600, 'sunrise': 1649826862, 'sun...","{'dt': 1649934000, 'sunrise': 1649913155, 'sun...","{'dt': 1650020400, 'sunrise': 1649999449, 'sun...","{'dt': 1650106800, 'sunrise': 1650085744, 'sun...","{'dt': 1650193200, 'sunrise': 1650172039, 'sun..."
0,45.8992,6.1289,Europe/Paris,7200,"[{'sender_name': 'METEO-FRANCE', 'event': 'Mod...","{'dt': 1649588400, 'sunrise': 1649567987, 'sun...","{'dt': 1649674800, 'sunrise': 1649654278, 'sun...","{'dt': 1649761200, 'sunrise': 1649740570, 'sun...","{'dt': 1649847600, 'sunrise': 1649826862, 'sun...","{'dt': 1649934000, 'sunrise': 1649913155, 'sun...","{'dt': 1650020400, 'sunrise': 1649999449, 'sun...","{'dt': 1650106800, 'sunrise': 1650085744, 'sun...","{'dt': 1650193200, 'sunrise': 1650172039, 'sun..."
0,42.9455,1.4066,Europe/Paris,7200,"[{'sender_name': 'METEO-FRANCE', 'event': 'Mod...","{'dt': 1649588400, 'sunrise': 1649567987, 'sun...","{'dt': 1649674800, 'sunrise': 1649654278, 'sun...","{'dt': 1649761200, 'sunrise': 1649740570, 'sun...","{'dt': 1649847600, 'sunrise': 1649826862, 'sun...","{'dt': 1649934000, 'sunrise': 1649913155, 'sun...","{'dt': 1650020400, 'sunrise': 1649999449, 'sun...","{'dt': 1650106800, 'sunrise': 1650085744, 'sun...","{'dt': 1650193200, 'sunrise': 1650172039, 'sun..."


In [16]:
def explode2_df_fn(col_name,df_exploded):
    #Subset first columns
    df_explode2=df_exploded.iloc[:,0:4]
    #Explode df
    df_explode3=df_explode2.join(pd.json_normalize(df_exploded[col_name]))
    df_explode3['Day']=col_name
    return(df_explode3)

explode2_df_fn("Today", df_explode)

,lat,lon,timezone,timezone_offset,dt,sunrise,sunset,moonrise,moonset,moon_phase,...,temp.min,temp.max,temp.night,temp.eve,temp.morn,feels_like.day,feels_like.night,feels_like.eve,feels_like.morn,Day
0,45.3314,1.0301,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,43.5298,5.4475,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,49.8987,2.2725,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,45.8992,6.1289,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,42.9455,1.4066,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,43.9492,4.8059,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,49.2765,-0.7025,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,43.4933,-1.4751,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,-33.9659,25.508,Africa/Johannesburg,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today
0,43.4711,-1.5527,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,1.02,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today


In [18]:
# Second data explosion: explode weather info for every day
names_7days=[str(df_explode.columns[x]) for x in range(6,13)]
td=["Today"]
days_ls=td+names_7days

weather_perday_df = explodeall_df_fn(days_ls, df_exploded=df_explode)
weather_perday_df.head()

Explosion done for all: 8 days
Shape of df:(280, 32)
Verify unique days: ['Today' 'Day_1' 'Day_2' 'Day_3' 'Day_4' 'Day_5' 'Day_6' 'Day_7']


,lat,lon,timezone,timezone_offset,dt,sunrise,sunset,moonrise,moonset,moon_phase,...,temp_max,temp_night,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,Day,rain
0,45.3314,1.0301,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today,NaN
0,43.5298,5.4475,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today,NaN
0,49.8987,2.2725,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today,NaN
0,45.8992,6.1289,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today,NaN
0,42.9455,1.4066,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,16.15,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today,NaN


In [67]:
# Export weather df
weather_perday_df.to_csv('data/processed/top_35_cities_france_weather_7days.csv', index=False)

## 3. Final dataset: Expanded weather data for 7 days for 35 cities

City Name included with lat and lon  
1 row equivalent to 1 day weather info for a specific city

In [68]:
# Merge GPS_df to weather_perday_df to get name of city

# Before merging: update type of columns in GPS df
GPS_df['lat']=pd.to_numeric(GPS_df["lat"])
GPS_df['lon']=pd.to_numeric(GPS_df["lon"])

# Round lat and lon to signif figures for merging
GPS_rounded_df=GPS_df.copy()
GPS_rounded_df['lat']=[np.round(num,4) for num in GPS_rounded_df['lat']]
GPS_rounded_df['lon']=[np.round(num,4) for num in GPS_rounded_df['lon']]

GPS_rounded_df.to_csv('data/processed/top_35_cities_france_gpscoord.csv', index=False)

GPS_rounded_df.head()

,Search_Name,lat,lon
1,Aigues Mortes,45.3314,1.0301
0,Aix-en-Provence,43.5298,5.4475
1,Amiens,49.8987,2.2725
0,Annecy,45.8992,6.1289
0,Ariège,42.9455,1.4066


In [27]:
# Final, merged dataframe. Lost info for Collioure despite updating on latitude and longitude 
gps_7dweather_df=weather_perday_df.merge(GPS_rounded_df,left_on=["lat","lon"],right_on=["lat","lon"])
print(gps_7dweather_df.shape)
print('{} unique cities: {}'.format(gps_7dweather_df.Search_Name.nunique(), gps_7dweather_df.Search_Name.unique()))
gps_7dweather_df.head()

(272, 33)
34 unique cities: ['Aigues Mortes' 'Aix-en-Provence' 'Amiens' 'Annecy' 'Ariège' 'Avignon'
 'Bayeux' 'Bayonne' 'Besancon' 'Biarritz' 'Bormes-les-Mimosas'
 'Carcassonne' 'Cassis' 'Château du Haut-Kœnigsbourg' 'Colmar' 'Dijon'
 'Eguisheim' 'Gorges du Verdon' 'Grenoble' 'La Rochelle' 'Le Havre'
 'Lille' 'Lyon' 'Marseille' 'Mont Saint Michel' 'Montauban' 'Nîmes'
 'Paris' 'Rouen' 'Saint-Malo' 'Saintes-Maries-de-la-Mer' 'Strasbourg'
 'Toulouse' 'Uzès']


,lat,lon,timezone,timezone_offset,dt,sunrise,sunset,moonrise,moonset,moon_phase,...,temp_night,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,Day,rain,Search_Name
0,45.3314,1.0301,Europe/Paris,7200,1649588400,1649567987,1649615654,1649589840,1649558640,0.28,...,9.96,12.32,1.02,12.08,8.96,11.15,-0.95,Today,NaN,Aigues Mortes
1,45.3314,1.0301,Europe/Paris,7200,1649674800,1649654278,1649702130,1649680200,1649647080,0.32,...,12.35,16.73,8.05,16.11,11.10,15.95,5.44,Day_1,NaN,Aigues Mortes
2,45.3314,1.0301,Europe/Paris,7200,1649761200,1649740570,1649788607,1649770800,1649735160,0.35,...,12.70,14.65,10.94,16.61,12.01,14.29,9.66,Day_2,0.15,Aigues Mortes
3,45.3314,1.0301,Europe/Paris,7200,1649847600,1649826862,1649875084,1649861400,1649823000,0.38,...,10.42,10.74,10.96,11.64,10.02,10.38,10.38,Day_3,2.34,Aigues Mortes
4,45.3314,1.0301,Europe/Paris,7200,1649934000,1649913155,1649961561,1649952120,1649910660,0.41,...,13.58,16.08,10.33,20.48,13.47,16.04,9.95,Day_4,2.72,Aigues Mortes


In [66]:
# Export final dataframe with weather for 7 days for top french cities to visit
gps_7dweather_df.to_csv('results/top_35_cities_france_weather_gps.csv', index=False)